In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Project/csv_files/updated_text_analysis.csv')
df.head(5)

,document,summary,id,rg_labels,ext_labels,document_sentiment,summary_sentiment,document_token,summary_token,document_topics,summary_topics
0,hi i I am getting a new laptop i do nt know wh...,of my post is asking what can i do compensate ...,TLDR_RS_2021-04-cm-17760.json,[1.0],[1],Positive,Positive,"['hi', 'i', 'I', 'am', 'getting', 'a', 'new', ...","['of', 'my', 'post', 'is', 'asking', 'what', '...","""i"" + ""."" + ""the"" + ""to"" + "",""",""","" + ""to"" + ""."" + ""a"" + ""you"""
1,it 's possible that many of you already do thi...,if you play hearthstone on mac and it runs poo...,TLDR_RS_2021-02-cm-3714.json,"[0.16403576510432702, 0.35967370810124305, 0.2...","[0, 1, 0, 0]",Positive,Positive,"['it', ""'s"", 'possible', 'that', 'many', 'of',...","['if', 'you', 'play', 'hearthstone', 'on', 'ma...","""i"" + ""."" + ""the"" + ""to"" + "",""","""."" + ""the"" + "":"" + ""i"" + ""to"""
2,"i will be the first to admit , i 've got a wei...",i 'm tired of my boyfriend bringing up my weig...,TLDR_RS_2021-04-cm-1732.json,"[0.1865050509103, 0.389592934910708, 0.2526842...","[0, 1, 0, 0, 0]",Positive,Negative,"['i', 'will', 'be', 'the', 'first', 'to', 'adm...","['i', ""'m"", 'tired', 'of', 'my', 'boyfriend', ...","""he"" + ""him"" + ""."" + ""his"" + ""to""","""i"" + ""."" + ""to"" + ""and"" + ""my"""
3,"hello , my friend 22m broke up a few months ag...",: my friend has suddenly stopped talking to me...,TLDR_RS_2021-03-cm-34926.json,"[0.13981801700034802, 0.054837487497308005, 0....","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",Negative,Neutral,"['hello', ',', 'my', 'friend', '22m', 'broke',...","[':', 'my', 'friend', 'has', 'suddenly', 'stop...","""i"" + ""."" + ""and"" + "","" + ""to""","""i"" + ""."" + ""to"" + ""and"" + ""my"""
4,background : my 18f ex boyfriend 18m really wa...,my ex is very erratic and depressed and never ...,TLDR_RS_2021-02-cm-18586.json,"[0.18652821177119003, 0.149288690725626, 0.038...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",Negative,Negative,"['background', ':', 'my', '18f', 'ex', 'boyfri...","['my', 'ex', 'is', 'very', 'erratic', 'and', '...","""i"" + ""."" + ""and"" + "","" + ""to""","""i"" + ""."" + ""to"" + ""and"" + ""my"""


In [ ]:
from sklearn.model_selection import train_test_split
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
import torch

# Set a fixed random seed for reproducibility
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

# Step 2: Select a smaller subset of the dataset
subset_size = 5000  # Choose the size of your subset
subset_df = df.sample(n=subset_size, random_state=random_seed)  # Randomly select records

# Step 3: Split the dataset into training and testing sets
train_df, test_df = train_test_split(subset_df, test_size=0.2, random_state=random_seed)

In [ ]:
# Step 4: Load the BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Function to tokenize the text data
def tokenize_function(examples):
    documents = examples['document'].astype(str).tolist()
    summaries = examples['summary'].astype(str).tolist()

    # Tokenize the documents and summaries
    model_inputs = tokenizer(documents, max_length=1024, truncation=True, padding=True)
    labels = tokenizer(summaries, max_length=150, truncation=True, padding=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Tokenize the training and test data
train_encodings = tokenize_function(train_df)
test_encodings = tokenize_function(test_df)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

In [ ]:
# Step 5: Prepare the data for the Trainer
class SummaryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = SummaryDataset(train_encodings)
test_dataset = SummaryDataset(test_encodings)

In [ ]:
# Step 6: Load the BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Step 7: Freeze layers
# Freeze the first 9 layers of the encoder
for layer in model.model.encoder.layers[:9]:  # Access encoder through model.model
    for param in layer.parameters():
        param.requires_grad = False

# Freeze the first 9 layers of the decoder
for layer in model.model.decoder.layers[:9]:  # Access decoder through model.model
    for param in layer.parameters():
        param.requires_grad = False

# Step 8: Define training arguments with mixed precision
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision training
)

# Step 9: Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Step 10: Train the model
trainer.train()

# Optional: Evaluate the model (if needed)
results = trainer.evaluate()
print(results)

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.049700,0.972282
2,0.836000,0.966887
3,0.700600,1.005086


{'eval_loss': 1.0050857067108154, 'eval_runtime': 40.0297, 'eval_samples_per_second': 24.981, 'eval_steps_per_second': 6.245, 'epoch': 3.0}


In [ ]:
# Save the model
model.save_pretrained('/content/drive/MyDrive/Project/model_files/bart_9E9D_model')

# Save the tokenizer
tokenizer.save_pretrained('/content/drive/MyDrive/Project/model_files/bart_9E9D_tokenizer')

('/content/drive/MyDrive/Project/model_files/bart_9E9D_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Project/model_files/bart_9E9D_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Project/model_files/bart_9E9D_tokenizer/vocab.json',
 '/content/drive/MyDrive/Project/model_files/bart_9E9D_tokenizer/merges.txt',
 '/content/drive/MyDrive/Project/model_files/bart_9E9D_tokenizer/added_tokens.json')